# SSMR: generate SSM model from trajectory data

In [ ]:
import numpy as np
from scipy.io import loadmat
from copy import deepcopy
import os

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [ ]:
np.set_printoptions(linewidth=300)

In [ ]:
use_data = "mine" # "mine", "john_svd_already_done", "john_full"
observables = "pos-vel" # "delay embedding"
TIP_NODE = 1354

## Import and inspect data
See mat file in URL

In [ ]:
Data = {}
if use_data == "john_svd_already_done":
    dataDecayObs = loadmat("../dataDecayObs.mat") # , simplify_cells=True)
    Data['oData'] = dataDecayObs['oData'].tolist()
    Data['etaData'] = dataDecayObs['yData'].tolist()
    # Xsnapshots = dataDecayObs['Xsnapshots']
    for i in range(len(Data['oData'])):
        Data['oData'][i][0] = Data['oData'][i][0].squeeze()
        # Data['yData'][i][0] = Data['yData'][i][0].squeeze()
elif use_data == "john_full":
    pass
elif use_data == "mine":
    diamond_dir = "../../../soft-robot-control/examples/diamond"
    decayData_dir = os.path.join(diamond_dir, "dataCollection/origin")
    rest_file = os.path.join(diamond_dir, 'rest.pkl')
    Data['oData'] = utils.import_pos_data(decayData_dir, rest_file, output_node='all', t_in=2, t_out=5)

rDOF = 3
oDOF = 3
nTRAJ = len(Data['oData'])
SSMDim = 6

print("nTRAJ:", nTRAJ)

In [ ]:
if use_data == 'john':
    for i in range(nTRAJ):
        Data['oData'][i][0] = Data['oData'][i][0][::10]
        Data['oData'][i][1] = Data['oData'][i][1][:, ::10]

## Observables

In [ ]:
observables = "pos-vel"

EITHER compute reduced coordinates using delay embedding on the available oData

In [ ]:
if observables == "delay embedding":
    N_DELAY = 4
    Data['yData'] = deepcopy(Data['oData'])
    for i in range(nTRAJ):
        delay_embedded_traj = utils.delayEmbedding(Data['oData'][i][1], up_to_delay=N_DELAY)
        Data['yData'][i][1] = delay_embedded_traj[:, :-4]
        Data['yData'][i][0] = Data['yData'][i][0][4:]

OR use position and velocity of all nodes as observables

In [ ]:
if observables == "pos-vel":
    Data['yData'] = deepcopy(Data['oData'])
    for i in range(nTRAJ):
        # observables y are pos and vel of tip
        Data['yData'][i][1] = np.vstack((Data['oData'][i][1][3*TIP_NODE:3*TIP_NODE+3, :], np.gradient(Data['oData'][i][1][3*TIP_NODE:3*TIP_NODE+3, :], 0.01, axis=1)))
        # do SVD on full state + full velocity
        # Data['oData'][i][1] = np.vstack((Data['oData'][i][1], np.gradient(Data['oData'][i][1], axis=1)))
    print(Data['yData'][0][1].shape)

### Plot observables of interest

In [ ]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = [] # [14, 16, 18, 31, 35, 44] # []
plot.traj_xyz(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
plot.traj_3D(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[0]+oDOF), ('yData', outdofs[2])],
             xyz_names=[r'$x$', r'$\dot{x}$', r'$z$'])

In [ ]:
t_interval = [0.08, 2.6]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

In [ ]:
# print(Data['yDataTrunc'][0][1].shape)
# Data['yDataTrunc'][0][1][:, :5]

## Obtain reduced-order coordinates

### Perform PCA on delay-embedded coordinates to obtain reduced coordinates

In [ ]:
# Perform SVD on displacement field
from scipy.sparse.linalg import svds

if observables == "pos-vel":
    svd_data = 'oDataTrunc'
elif observables == "delay embedding":
    svd_data = 'yDataTrunc'

Xsnapshots = np.hstack([DataTrunc[1] for DataTrunc in Data[svd_data]])
Xsnapshots = Xsnapshots[:, :]

# print(Xsnapshots.shape)
# print(Xsnapshots[:, 0:10])

show_modes = 9
v, s, u = svds(Xsnapshots, k=max(SSMDim, show_modes), which="LM")
ind = np.argsort(s)[::-1]
s = s[ind]
v = v[:, ind]
# v, s, _ = np.linalg.svd(Xsnapshots, compute_uv=True)
# print(s)
# print(v)
l2vals = s**2

In [ ]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=9)

### Project delay-embedded data onto dominant modes

In [ ]:
from copy import deepcopy
Data['etaDataTrunc'] = deepcopy(Data[svd_data])
if observables == "delay embedding":
    Vde = v[:, :SSMDim]
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ Data[svd_data][i][1]
elif observables == "pos-vel":
    assert SSMDim % 2 == 0
    Vde = np.kron(np.eye(2), v[:, :SSMDim//2])
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ np.vstack((Data['oDataTrunc'][i][1], np.gradient(Data['oDataTrunc'][i][1], 0.01, axis=1))) # np.vstack((np.gradient(Data['oData'][i][1], axis=1), Data['oData'][i][1])) # Vde.T @ np.vstack((Data[svd_data][i][1], np.gradient(Data[svd_data][i][1], axis=1))) # 

In [ ]:
plt.close('all')
# plot first three reduced coordinates
plot.traj_3D(Data,
              xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 3), ('etaDataTrunc', 1)],
              xyz_names=[r'$x_1$', r'$\dot{x_1}$', r'$x_2$'])

### Train and test data (train/test split)

In [ ]:
indTest = [0, 2, 4, 6, 8, 10, 12, 14]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

## SSMLearn

Start Matlab engine and install/run SSMLearn

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [ ]:
# make data ready for Matlab
yDataTruncTrain_matlab = [[], []]
etaDataTruncTrain_matlab = [[], []]
for i in indTrain:
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][i][0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][i][1].tolist()))
    etaDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['etaDataTrunc'][i][0].tolist()))
    etaDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['etaDataTrunc'][i][1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [ ]:
SSMOrder = 3
obsDim = 2*oDOF

IMInfo = eng.IMGeometry(yDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', etaDataTruncTrain_matlab) # , 'l_vals', 1.)
if observables == "pos-vel":
    IMInfoInv = eng.IMGeometry(etaDataTruncTrain_matlab, obsDim, SSMOrder,
                            'reducedCoordinates', yDataTruncTrain_matlab)
    for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
        IMInfo['chart'][key] = IMInfoInv['parametrization'][key]


### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [ ]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(etaDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default')

Stop Matlab engine -- not needed anymore

In [ ]:
eng.quit()

Convert all matlab double arrays to numpy arrays

In [ ]:
def convert_matlab_arrays_to_numpy_arrays(info_dict):
  for k,v in info_dict.items():        
     if isinstance(v, dict):
         convert_matlab_arrays_to_numpy_arrays(v)
     else:            
        if isinstance(v, matlab.double):
           info_dict[k] = np.array(v)

convert_matlab_arrays_to_numpy_arrays(IMInfo)
convert_matlab_arrays_to_numpy_arrays(RDInfo)

Stability analysis of reduced dynamics

In [ ]:
assert np.all(np.real(RDInfo['eigenvaluesLinPartFlow']) < 0)
RDInfo['eigenvaluesLinPartFlow']

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [ ]:
yRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [ ]:
yRec['Train'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTrain])
errorGeo['Train'] = utils.compute_trajectory_errors(yRec['Train'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4e}")

#### Test error

In [ ]:
yRec['Test'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTest])
errorGeo['Test'] = utils.compute_trajectory_errors(yRec['Test'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4e}")

Plot comparison of SSM-predicted vs. actual test trajectories

In [ ]:
print(Data['yData'][0][1].shape)
print(yRec['Test'][0][1].shape)

In [ ]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [ ]:
etaRec = {}
etaRec['RD'] = utils.advectRD(RDInfo, Data['etaDataTrunc'])[0]
meanErrorDyn = {}

for i in range(nTRAJ):
    if etaRec['RD'][i][1].shape[1] != Data['etaDataTrunc'][i][1].shape[1]:
        print(i)

normedTrajDist = utils.compute_trajectory_errors(etaRec['RD'], Data['etaDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

In [ ]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(etaRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [ ]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [ ]:
yRec['RDTrain'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTrain])
errorGlo['Train'] = utils.compute_trajectory_errors(yRec['RDTrain'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

#### Test error

In [ ]:
yRec['RDTest'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTest])
errorGlo['Test'] = utils.compute_trajectory_errors(yRec['RDTest'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

In [ ]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

## Control

### Setup model for control

In [ ]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d 

In [ ]:
Rauton = utils.Rauton(RDInfo)
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])
if observables == "pos-vel":
    Wauton = lambda y: IMInfo['chart']['H'] @ utils.multivariate_polynomial(y, IMInfo['chart']['polynomialOrder'])
elif observables == "delay embedding":
    Wauton = lambda y: Vde.T @ y

### Learn control matrix $B$

Use figure8 trajectory as the training data

In [ ]:
# # Load the controls and actual trajectories for figure8
# u_train = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
# z_train = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
# z_eq_train = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
# print(z_eq_train)

# # tip position, normalized using resting position
# z_shift_train = (z_train[-3:, :].T - z_eq_train).T

# # original spacing of inputs
# T = 10.01
# dt = 0.01
# N = int(T / dt)
# t_train = np.linspace(0, T, N+1)

Use randomly sampled (step) inputs as the training data

In [ ]:
# diamond_train_full = loadmat("../diamond_train_full.mat", simplify_cells=True)
# z_train, u_train, _ = diamond_train_full['y'].T, diamond_train_full['u'].T, diamond_train_full['t']
# print(z_train.shape)

# # tip position, normalized using resting position
# z_shift_train = (z_train[-3:, :].T - z_train[-3:, 0]).T
# dt = 0.01
# t_train = np.arange(z_train.shape[1]) * dt

Use randomly sampled inputs as the training and test data

In [ ]:
inputData_dir = os.path.join(diamond_dir, "dataCollection/controlled")
(t, z_shift), u = utils.import_pos_data(inputData_dir, rest_file, TIP_NODE, return_inputs=True)
print(z_shift.shape)
# print(u_train)
dt = 0.01

Train/test split

In [ ]:
train_ratio = 0.8
split_idx = int(train_ratio * len(t))
t_train, t_test = t[:split_idx], t[split_idx:]
z_shift_train, z_shift_test = z_shift[:, :split_idx], z_shift[:, split_idx:]
u_train, u_test = u[:, :split_idx], u[:, split_idx:]

Plot the training data

In [ ]:
plt.close('all')
plot.traj_xyz_txyz(t=t_train,
                   x=z_shift_train[0, :], y=z_shift_train[1, :], z=z_shift_train[2, :])
plot.inputs(t_train, u_train)

Plot the test data

In [ ]:
plt.close('all')
plot.traj_xyz_txyz(t=t_test,
                   x=z_shift_test[0, :], y=z_shift_test[1, :], z=z_shift_test[2, :])
plot.inputs(t_test, u_test)

Do delay embedding on the tip position. Then, compute reduced coordinates and gradient as predicted by reduced dynamics, as well as the numerical (true) gradient

In [ ]:
if observables == "delay embeddings":
    # delay-embed tip position
    y_train = utils.delayEmbedding(z_shift_train, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_train = np.vstack((z_shift_train, np.gradient(z_shift_train, 0.01, axis=1)))

print(f"y.shape: {y_train.shape}, u.shape: {u_train.shape}")
# reduced coordinates
x_train = Wauton(y_train)
dxDt = np.gradient(x_train, dt, axis=1)
dxDt_ROM = Rauton(x_train)

# optionally, throw away some of the training data
keep_indices = np.s_[10:-10]
t_train_trunc = t_train[keep_indices]
u_train_trunc = u_train[:, keep_indices]
x_train_trunc = x_train[:, keep_indices]
y_train_trunc = y_train[:, keep_indices]
dxDt = dxDt[:, keep_indices]
dxDt_ROM = dxDt_ROM[:, keep_indices]

print(t_train_trunc.shape)

In [ ]:
plt.close('all')
ax = plot.traj_3D_xyz(y_train_trunc[outdofs[0], :],
                      y_train_trunc[outdofs[1], :],
                      y_train_trunc[outdofs[2], :], show=True)

Plot gradients (numerical differentiation and reduced dynamics)

In [ ]:
print(dxDt.shape)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt, dxDt_ROM], labels=["true numerical", "predicted autonomous"], how="all")

In [ ]:
plt.close('all')
plot.dependence_of_xdot_on_inputs(dxDt, u_train_trunc)

Learn $B$ matrix

In [ ]:
from sklearn.linear_model import Ridge

polyUorder = 1
# assemble_features = lambda u, x: utils.multivariate_polynomial(np.vstack([u, x]), order=polyUorder)
assemble_features = lambda u, x: utils.multivariate_polynomial(u, order=polyUorder)

X = assemble_features(u_train_trunc, x_train_trunc)
print(X.shape)

ridgeModel = Ridge(alpha=0, fit_intercept=False)
ridgeModel.fit(X.T, (dxDt - dxDt_ROM).T)
B_learn = ridgeModel.coef_
print(B_learn.shape)

Frobenius norm of $B_{learn}$ to estimate how much influence inputs have on dynamics

In [ ]:
print(f"Frobenius norm of B_learn: {np.linalg.norm(B_learn, ord='fro'):.4f}")

Plot gradients again, but this time, using the learned influence of inputs for the prediction

In [ ]:
plt.close('all')
plot_reduced_coords = np.s_[:] # [3, 4, 5]
dxDt_ROM_with_B = Rauton(x_train_trunc) + B_learn @ assemble_features(u_train_trunc, x_train_trunc) # utils.multivariate_polynomial(u_train_trunc, order=polyUorder)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt[plot_reduced_coords, :], dxDt_ROM[plot_reduced_coords, :], dxDt_ROM_with_B[plot_reduced_coords, :]], labels=["true numerical", "predicted autonomous", "predicted with inputs"], how="all")

In [ ]:
# # home-brewed ridge regression
# alpha = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + alpha * np.eye(X.shape[0]), X.T @ y)

### Integrate model with inputs (big amplitude) -- and using learned influence of control

Try to predict the test trajectory using reduced dynamics

In [ ]:
if observables == "delay embeddings":
    # delay-embed tip position
    y_test = utils.delayEmbedding(z_shift_test, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_test = np.vstack((z_shift_test, np.gradient(z_shift_test, 0.01, axis=1)))

# reduced coordinates
x_test = Wauton(y_test)

slice_traj = np.s_[:]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_shift_test = z_shift_test[:, slice_traj]

In [ ]:
# Load the controls and actual trajectories for figure8
u_test = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_test = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_test = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_test.shape)
# tip position, normalized using resting position
z_shift_test = (z_test[-3:, :].T - z_eq_test).T
# delay-embed tip position (as well as inputs?)
if observables == "delay embeddings":
    # delay-embed tip position
    y_test = utils.delayEmbedding(z_shift_test, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
elif observables == "pos-vel":
    y_test = np.vstack((z_shift_test, np.gradient(z_shift_test, 0.01, axis=1)))# reduced coordinates
x_test = Wauton(y_test)

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_test = np.linspace(0, T, N+1)

slice_traj = np.s_[:]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_test = z_test[:, slice_traj]

In [ ]:
uInterpFun = interp1d(t_test, u_test, axis=1, fill_value="extrapolate")
uFun = lambda t: uInterpFun(t).reshape(-1, 1)
# print(t_test.shape, dxDt.shape)
# dxDtInterpFun = interp1d(t_test, dxDt_ROM, axis=1, fill_value="extrapolate")
def R(t, y):
    return Rauton(np.atleast_2d(y)) + B_learn @ assemble_features(uFun(t), y) # utils.multivariate_polynomial(uFun(t), order=polyUorder)
    # return dxDtInterpFun(t)

# solve IVP of reduced dynamics using open-loop figure8 inputs
sol = solve_ivp(R,
                t_span=[t_test[0], t_test[-1]],
                t_eval=t_test,
                y0=x_test[:, 0], # np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
# resulting (predicted) open-loop trajectory in reduced coordinates
xTraj = sol.y
yTraj = Vauton(xTraj)
zTraj = yTraj[:3, :] # .T + z_eq_test).T
print(zTraj.shape)

plt.close('all')
axs = plot.traj_xyz_txyz(t_test,
                    zTraj[outdofs[0], :],
                    zTraj[outdofs[1], :],
                    zTraj[outdofs[2], :], show=False)
# ax = plot.traj_3D_xyz(z_test[3, :],
#                       z_test[4, :],
#                       z_test[5, :], color="tab:orange", ax=ax, show=False)
axs = plot.traj_xyz_txyz(t_test,
                    z_shift_test[outdofs[0], :],
                    z_shift_test[outdofs[1], :],
                    z_shift_test[outdofs[2], :], color="tab:orange", axs=axs, show=False)
axs[2].legend(["Predicted trajectory", "Actual trajectory"])
plt.show()


In [ ]:
RMSE = np.sum(np.sqrt(np.mean((zTraj[0:3, :] - z_shift_test[-3:])**2, axis=0))) / zTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")

## Save SSM model

Continuous model

In [ ]:
SSM_model = {'model': {}, 'params': {}}

print(IMInfo['parametrization']['H'].shape)
print(IMInfo['chart']['H'].shape)
SSM_model['model']['w_coeff'] = IMInfo['parametrization']['H']
SSM_model['model']['v_coeff'] = IMInfo['chart']['H']
SSM_model['model']['r_coeff'] = RDInfo['reducedDynamics']['coefficients']
SSM_model['model']['B'] = B_learn
SSM_model['model']['V'] = None # Vde

SSM_model['params']['SSM_order'] = SSMOrder
SSM_model['params']['ROM_order'] = ROMOrder
SSM_model['params']['state_dim'] = SSMDim # 2 * rDOF
print(SSMDim)
# print(2*rDOF)
SSM_model['params']['input_dim'] = B_learn.shape[1]
SSM_model['params']['output_dim'] = oDOF * 2

if observables == "delay embeddings":
    SSM_model['params']['delays'] = N_DELAY
    SSM_model['params']['obs_dim'] = 3 * (1 + N_DELAY)
elif observables == "pos-vel":
    SSM_model['params']['delays'] = 0
    SSM_model['params']['obs_dim'] = 6
# print(Data['yData'][0][1].shape[0])

In [ ]:
import pickle
# save dictionary to person_data.pkl file
with open('../../../soft-robot-control/examples/hardware/SSMmodels/SSM_model.pkl', 'wb') as f:
    pickle.dump(SSM_model, f)